# Documentation: Tag a Turtle – Video Segmentation Pipeline with YOLOv11


---
| Author                   | Date        | Version      |
|--------------------------|-------------|--------------|
| Yasmin Abhrao Kfuri      | 25/06/2025  | 1.0          |



---


## Table of Contents

1. [Project Overview - Summary](#1)
2. [Introduction](#2)
3. [Pipeline Analysis](#3)
4. [Justification](#4)
5. [Pipeline Design](#5)
6. [Installing Dependencies](#6)
7. [Importing Files](#7)
8. [Realizationg of segmenting the video](#8)
9. [Manage and Control Considerations](#10)
10. [Conclusion](#11)
11. [Sources](#12)



---



## Project Overview - Summary <a name="1"></a>

Tag a Turtle is an AI-powered computer vision project for detecting, segmenting and identifying sea turtles in under and above water footage. In this document, the designed pipeline supports marine biodiversity research by identifying individuals based on their physical characteristics (e.g., head and flipper markings) using Yolov11 instance segmentation model trained in the previous assignment *Yolov11 Instance Segmentation Model with Custom Data for Tag a Turtle Project* and testing it on videos.


## Introduction <a name="2"></a>

The Tag a Turtle project focuses on the development of a video segmentation pipeline aimed at assisting sea turtle conservation efforts. The pipeline automates the identification of sea turtles and their body parts — such as head and flippers — within underwater videos. This facilitates individual recognition for tracking and research purposes.

To support this, the pipeline integrates YOLOv11 (You Only Look Once, version 11), a real-time object detection and segmentation model. YOLOv11 builds upon the strengths of earlier YOLO architectures, offering fast and accurate detection alongside pixel-level segmentation masks. These features make it particularly suitable for complex underwater scenes where lighting, motion, and occlusion present challenges for traditional computer vision methods.

The resulting system is deployed in Google Colab and processes videos uploaded by the user. It performs object detection and segmentation frame-by-frame, annotates results, and compiles them back into downloadable video files.

Attached to this document, there are the outputs (videos) where the results can be seen.

## Pipeline Analysis <a name="3"></a>

The pipeline was developed to automate object detection and segmentation in underwater turtle videos. The core functionality was built around the YOLOv11 model, selected for its ability to perform real-time detection and instance segmentation. The model accepts single frames extracted from videos and returns bounding boxes, labels, and segmentation masks for each detected object.

The software solution was implemented in Python and designed to operate within the Google Colab environment - where the stakeholder virtually operates. This choice addressed hardware limitations while providing ease of access and rapid prototyping. The pipeline makes use of modular blocks: one for model loading (YOLO(pt_file)), another for video decoding using OpenCV `(cv2.VideoCapture())`, followed by a frame processing loop where `model(frame)` is executed and the results are visualized with `results[0].plot()`.

By structuring the pipeline in this way, each component could be independently analyzed and debugged. Problems such as inconsistent frame sizes, slow frame rates, or memory overflows were mitigated through frame-by-frame inference, batch video handling, and format checks.

## Justification <a name="4"></a>

The pipeline design was adapted to function within the constraints of Colab and the variability of uploaded user content. Instead of relying on predefined file paths, the system uses files.upload() to allow dynamic model and video input. The code automatically filters .pt files for model loading and `.mp4`, `.mov`, or `.MOV` files for video input. This ensures that only relevant files are processed and reduces the risk of input errors.

Batch processing was implemented by looping through all uploaded video files. For each video, OpenCV reads frames in sequence, the YOLOv11 model performs detection and segmentation, and annotated frames are compiled using cv2.VideoWriter. These design choices simplify user interaction and allow the same pipeline to handle multiple files without reconfiguration.

Adjustments were made to handle environmental unpredictability. For instance, fallback logic is included to prompt for manual model file entry if automatic detection fails. Output filenames are generated based on original inputs to prevent overwriting and ensure traceability.

## Pipeline Design <a name="5"></a>

The pipeline was structured for compatibility with existing ICT systems and intended for integration with typical file storage services such as Google Drive. The use of standard video formats `(.mp4, .mov)` and the output of annotated results in `.mp4` format ensure that results are readily usable in video editors, research archives, or communication tools.

A minimal set of dependencies — primarily ultralytics and opencv-python — was selected to reduce complexity. The system requires no installation beyond pip commands and runs entirely in the cloud. This supports reproducibility and makes it feasible to onboard non-technical users into the workflow.

To manage design unpredictability, only necessary parameters were exposed to users. The confidence threshold and overlap parameters from YOLOv11 are adjustable if needed, but the default settings are optimized for reliable performance in general use cases. This balances design efficiency with flexibility.

### Installing Dependencies <a name="6"></a>



In [ ]:
!pip install ultralytics opencv-python

## Importing files <a name="7"></a>

When using Google Colab, you can either mount the drive and access the files there, or uploading via Colab. In this case, we're uploading both the video we want to segment and the model yolov11 instance segmentation with custom data.

In [ ]:
from google.colab import files
uploaded = files.upload()

If mounting the drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Realizationg of segmenting the video <a name="8"></a>
The video segmentation pipeline was implemented in discrete modules to simplify development and testing. The system loads the trained YOLOv11 model from a .pt file using the YOLO() class. Uploaded videos are processed using OpenCV, and frame-level inference is executed within a loop until all frames have been analyzed.

During each iteration, the model performs detection and segmentation using model(frame). The first result in the output is plotted directly onto the image with results[0].plot() to produce bounding boxes and segmentation masks in a unified frame. This method ensures that each output video reflects not just turtle presence but their physical segmentation, necessary for biometric matching.

Test videos of varying resolution and format were used to verify the robustness of the pipeline. Success was defined by complete video processing without errors, appropriate overlay of annotations, and compatibility of the output with media players and download tools in Colab.

In [ ]:
# Identify model and videos
import os

uploaded_files = list(uploaded.keys())
model_file = [f for f in uploaded_files if f.endswith('.pt')][0]
video_files = [f for f in uploaded_files if f.endswith(('.mp4', '.mov', '.MOV'))]

# Load the model
from ultralytics import YOLO

print(f"Loading model: {model_file}")
model = YOLO("yolov11-segmentation.pt")

# Process each video
import cv2

for video_path in video_files:
    print(f"Processing {video_path}...")

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Output file name
    output_path = f"annotated_{os.path.splitext(video_path)[0]}.mp4"
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    # Frame-by-frame processing
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        annotated = results[0].plot()
        out.write(annotated)

    cap.release()
    out.release()

    print(f"Saved: {output_path}")

# ⬇️ Download all processed videos
from google.colab import files

for video_path in video_files:
    output_path = f"annotated_{os.path.splitext(video_path)[0]}.mp4"
    files.download(output_path)

### Manage and Control Considerations <a name="10"></a>

The pipeline is designed for easy maintenance and performance monitoring. It operates within Google Colab’s environment, minimizing system compatibility concerns and leveraging cloud-based scalability. The annotated outputs are automatically named and stored, allowing them to be downloaded immediately after processing.

File filtering and fallback handling were included to manage unexpected user input. These controls reduce potential errors and make the pipeline more robust when shared with broader user groups. The modularity of the implementation also supports rapid updates; for example, changing the model version or adding new detection classes would require minimal adjustments.

The pipeline can be extended further to include turtle ID matching, trajectory tracking, or export to metadata-rich formats like JSON. Its current configuration provides a strong baseline that meets functional requirements while remaining flexible for future use in conservation, education, or AI research contexts.



## Conclusion <a name="11"></a>

The video segmentation pipeline built for the **Tag a Turtle** project demonstrates strong performance in identifying and segmenting sea turtles in varied underwater video conditions. The most critical detection class — the head, used for individual identification — achieved a segmentation accuracy ranging from 60% to 93%, which is highly promising given the challenging visual environments and the relatively short training duration of 100 epochs. These results suggest that even better accuracy could be achieved through further fine-tuning of the model, potentially by training for more epochs or improving annotation quality.

A key advantage of testing the model on video is the ability to visually evaluate the output over multiple frames and angles. This helps confirm the model's consistency in real-world scenarios, as segmentation quality remains stable despite motion, lighting changes, or brief occlusions.

During testing, three annotated videos were reviewed:

`annotated_video1 (1):` A clean, well-lit video sourced from Canva. Segmentation remains clear and consistent throughout the majority of the footage, showcasing the model’s ability to generalize to synthetic or controlled visuals.

`annotated_video2 (1):` Another Canva-based video. In this case, segmentation quality fluctuates slightly more, indicating that fine-tuning would significantly benefit the model. This may be due to subtle differences in visual style or object scale, which the model was less exposed to during training.

`annotated_video3 (1):` A real stakeholder-provided video captured in a natural environment. Despite erratic movements and surface reflections, the model performs exceptionally well — achieving segmentation accuracy for the turtle’s head as high as 93%. This highlights the pipeline's robustness, even under chaotic and unstructured video conditions.

Overall, the system presents a strong foundation for supporting sea turtle tracking via video analysis and can be further improved through targeted retraining and dataset expansion. Its integration in video workflows makes it a scalable and practical solution for conservationists and researchers alike.

## Sources <a name="12"></a>
- [Roboflow Documentation](https://docs.roboflow.com)
- [Ultralytics YOLOv8 Docs (YOLOv11-compatible syntax)](https://docs.ultralytics.com)
- [OpenCV Video I/O](https://docs.opencv.org)
- [Google Colab Documentation](https://colab.research.google.com)